# Mobile Games AB Testing with Cookie Cats

Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру. Mobile Games AB Testing with Cookie Cats

Данные для анализа: https://docs.google.com/spreadsheets/d/1Dj6c6pmdg1N_rL3T2zg4HxnBo7poBgm6-g-40ATIJmE/edit#gid=1733885895 ("результаты А_B - cookie_cats 2.csv")

### Начало

Импортируем библиотеки Python, необходимые для анализа данных

In [53]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from tqdm.notebook import tqdm

plt.style.use('ggplot')
%matplotlib inline

### Анализ данных 

Загружаем данные из CSV-файла "результаты А_B - cookie_cats 2.csv" и анализируем вид этих данных 

In [54]:
data = pd.read_csv("результаты А_B - cookie_cats 2.csv")

In [55]:
data.shape

(90189, 5)

In [56]:
data.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1


In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


In [58]:
data.retention_1.value_counts(), data.retention_7.value_counts()

(retention_1
 0    50036
 1    40153
 Name: count, dtype: int64,
 retention_7
 0    73408
 1    16781
 Name: count, dtype: int64)

По итогу видим, что данные содержат сравнение двух версий игры: _с воротами на уровне 30_ (gate_30) и _на уровне 40_ (gate_40). 
Основные метрики - количество игровых раундов (sum_gamerounds), удержание через 1 день (retention_1) и удержание через 7 дней (retention_7).

### Тесты

Для дальнейшего тестирования будем считать пользователей из gate_30 - контрольной выборкой, а из gate_40 - тестовой.
И проведёт тесты, сравнивающие основные метрики для теста и контроля.

В данных отсутствуют пользователи, находящиеся одновременно в двух выборках. А также каждая строчка данных соответствует ровно 1 пользователю.

In [59]:
data.groupby("userid")["version"].count().value_counts()

version
1    90189
Name: count, dtype: int64

In [60]:
control = data[data.version == "gate_30"]
test = data[data.version == "gate_40"]

In [61]:
control.shape, test.shape, control.shape[0] + test.shape[0]

((44700, 5), (45489, 5), 90189)

In [62]:
control_gamerounds = control.sum_gamerounds.values
test_gamerounds = test.sum_gamerounds.values

Начнём проводить тесты. Начнём с метрики количества игровых раундов.

In [63]:
from scipy.stats import ttest_ind
from scipy.stats import levene
from scipy.stats import mannwhitneyu

Проверим выборки на равенство дисперсий с помощью теста Левена.

In [64]:
levene(control_gamerounds, test_gamerounds)

LeveneResult(statistic=np.float64(0.5292002638313259), pvalue=np.float64(0.46694516772623273))

Поскольку p-value > 0.05, нулевая гипотеза (дисперсии равны) не отвергается. 
Это позволяет использовать t-тест с предположением о равных дисперсиях.

Но перед t-тестом проведём тест Манна-Уитни, который сравнивает распределения числа сыгранных раундов.

In [65]:
mannwhitneyu(control_gamerounds, test_gamerounds)

MannwhitneyuResult(statistic=np.float64(1024331250.5), pvalue=np.float64(0.05020880772044255))

P-value слегка выше 0.05, что указывает на отсутствие статистически значимых различий на уровне значимости 5%. 
То есть тест и контроль имеют одинаковые распределения.

Хотя данные и не должны быть распределены нормально, но с учётом размера наших выборок, можем провести t-тест для сравнения матожиданий выборок.

In [66]:
ttest_ind(control_gamerounds, test_gamerounds, equal_var=True)

TtestResult(statistic=np.float64(0.8910426211362967), pvalue=np.float64(0.37290868247405207), df=np.float64(90187.0))

P-value значительно выше 0.05, нулевая гипотеза (средние равны) не отвергается. Различий в среднем количестве раундов нет.

_Таким образом, сдвиг ворот с 30 на 40 уровень, существенно не сказался на количестве раундов, сыгранных пользователями._

Теперь проведём z-тесты для retention за 1 и 7 дни

In [84]:
control_retention_1 = control.retention_1.values
test_retention_1 = test.retention_1.values

control_retention_7 = control.retention_7.values
test_retention_7 = test.retention_7.values

Для проведения z-теста запишем функция ret_test для проверки статистической значимости различий в пропорциях

In [85]:
def ret_test(ma, na, mb, nb, alpha, alternative="two-sided"):
    '''
    output:
    False - H_0 принимается 
    True - H_0 отвергается 
    '''
    wa = ma / na
    wb = mb / nb
    w = (ma + mb) / (na + nb)
    diff = wb - wa
    SE = np.sqrt(w * (1-w) * (1/na + 1/nb))
    stat = (diff) / SE
    # 
    dist_norm = stats.norm(loc=0, scale=1)
    quantile = dist_norm.ppf(alpha/2)
    # 
    CI = (diff + SE * quantile, diff - SE * quantile)
    # 
    if alternative == "two-sided":
        p_value = 2 * min(dist_norm.sf(stat), dist_norm.cdf(stat))
        if p_value > alpha:
            return False, CI
        return True, CI
    elif alternative == "greater":
        p_value = dist_norm.sf(stat)
        if p_value > alpha:
            return False, CI
        return True, CI
    elif alternative == "less":
        p_value = dist_norm.cdf(stat)
        if p_value > alpha:
            return False, CI
        return True, CI
    return None, CI

Проверяем retention за 1 день

In [86]:
alpha = .05

ret_test(control_retention_1.sum(), control_retention_1.shape[0], test_retention_1.sum(), test_retention_1.shape[0], alpha, alternative="two-sided")

(False, (np.float64(-0.012392479618511289), np.float64(0.0005821400438283724)))

Результат: False (H₀ не отвергается), доверительный интервал включает 0 (-0.0124, 0.0006). Различий в удержании через 1 день нет.

Проверяем retention за 7 дней

In [87]:
ret_test(control_retention_7.sum(), control_retention_7.shape[0], test_retention_7.sum(), test_retention_7.shape[0], alpha, alternative="two-sided")

(True, (np.float64(-0.013281079012494897), np.float64(-0.0031215176179169284)))

In [88]:
ret_test(control_retention_7.sum(), control_retention_7.shape[0], test_retention_7.sum(), test_retention_7.shape[0], alpha, alternative="less")

(True, (np.float64(-0.013281079012494897), np.float64(-0.0031215176179169284)))

### Вывод

Таким образом, True — нулевая гипотеза (H₀: пропорции удержания в группах равны) отвергается на уровне значимости α = 0.05. Доверительный интервал (CI): от -0.01328 до -0.00312 (не включает 0).


Итак, Разница в удержании через 7 дней между тестовой (gate_40) и контрольной (gate_30) группами статистически значима.
Отрицательный интервал указывает, что удержание в тестовой группе ниже, чем в контрольной.

Итак, анализ A/B-теста Cookie Cats показал, что перемещение ворот с 30 на 40 уровень не изменило количество сыгранных раундов (t-тест: p-value = 0.373, U-тест: p-value = 0.0502, различий нет) и удержание через 1 день (z-тест: p-value > 0.05, CI включает 0), но привело к статистически значимому снижению удержания через 7 дней в тестовой группе (gate_40) по сравнению с контрольной (gate_30). Двусторонний z-тест подтвердил различие (H₀ отвергается, CI: -0.01328, -0.00312), а односторонний тест показал, что удержание в gate_40 ниже примерно на 0.82% (диапазон 0.31%–1.33%). Это говорит о том, что игроки в тестовой группе реже возвращаются через неделю, возможно, из-за более позднего появления ворот, что демотивирует их. Рекомендую оставить текущую версию gate_30, так как она обеспечивает лучшее удержание через 7 дней — ключевой показатель долгосрочной вовлечённости. Внедрение gate_40 может привести к потере игроков, и стоит рассмотреть другие варианты изменений или сегментацию данных для понимания причин спада.